 ### STEPS TO BE TAKEN : 
    Introduction and Importing the Data

    Transforming Documents into Feature Vectors

    Term Frequency-Inverse Document Frequency (TF-IDF)

    Calculate the TF-IDF of the Term 'Is'

    Data Preparation 

    Tokenization of Documents

    Document Classification Using Logistic Regression

    Load Saved Model from Disk

    Model Accuracy

#### Importing libraries and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df=pd.read_csv(r"C:\Users\himanshu pandey\Downloads\movie_data.csv")

In [3]:
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


#### The movie having atleast 7 stars can be labeled as positive
#### The movie with atmost 4 stars can be labeled as negative

In [4]:
df.describe()

,sentiment
count,50000.000000
mean,0.500000
std,0.500005
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

Total number of values = 50000, Total null values = 0

##### Reading a complete review : for example 6th review

In [14]:
df.loc[5][0]

"Leave it to Braik to put on a good show. Finally he and Zorak are living their own lives outside of Spac Ghost Coast To Coast. I have to say that I love both of these shows a whole lot. They are completely what started Adult Swim. Brak made it big with an album that came out in the year 2000. It may not have been platinum, but his show was really popular to tons of people out there that love Adult Swims shows. I have to say that out of all the Adult Swim shows with no plot, this has to be the one with the most none plot ever made. That is why I like it so much, it is just such a classic in the Adult Swim history. I believe this is just such a great show, if you don't like it. Hey there were tons who hated it and tons who loved it."

#### Transforming documents into feature vectors

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()

docs=np.array(['The sun is shining',
              'The weather is sweet',
              ' The sun is shining,The weather is sweet, and one and one is two'])


bag=count.fit_transform(docs)

In [36]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [37]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [38]:
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision=2)
tfidf = TfidfTransformer(use_idf = True, norm = 'l2',smooth_idf = True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


tfidf weights everything appropriately, from 3--> 0.45 weight is given to is

##### DATA PREPARATION

In [42]:
df.loc[0,'review']

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

In [63]:
t=df.loc[0,'review']

###### A lot of data seems irrelevant in the review, it must be removed.

In [64]:
import re
def preprocessor(text):
    text = re.sub ('<[^>]*>','',text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text=re.sub('[\W]+',' ',text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [65]:
preprocessor(t)

'in 1974 the teenager martha moxley maggie grace moves to the high class area of belle haven greenwich connecticut on the mischief night eve of halloween she was murdered in the backyard of her house and her murder remained unsolved twenty two years later the writer mark fuhrman christopher meloni who is a former la detective that has fallen in disgrace for perjury in o j simpson trial and moved to idaho decides to investigate the case with his partner stephen weeks andrew mitchell with the purpose of writing a book the locals squirm and do not welcome them but with the support of the retired detective steve carroll robert forster that was in charge of the investigation in the 70 s they discover the criminal and a net of power and money to cover the murder murder in greenwich is a good tv movie with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a kennedy the powerful and rich family used their influence to cover the mur

In [61]:
df['review']=df['review'].apply(preprocessor)

In [62]:
df.head()

,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0


###### Tokenization of documents

In [84]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [85]:
def tokenizer(text):
    return text.split()

In [86]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
        

In [87]:
tokenizer(df.loc[4,'review'])

['i',
 'recently',
 'bought',
 'the',
 'dvd',
 'forgetting',
 'just',
 'how',
 'much',
 'i',
 'hated',
 'the',
 'movie',
 'version',
 'of',
 'a',
 'chorus',
 'line',
 'every',
 'change',
 'the',
 'director',
 'attenborough',
 'made',
 'to',
 'the',
 'story',
 'failed',
 'by',
 'making',
 'the',
 'director',
 'cassie',
 'relationship',
 'so',
 'prominent',
 'the',
 'entire',
 'ensemble',
 'premise',
 'of',
 'the',
 'musical',
 'sails',
 'out',
 'the',
 'window',
 'some',
 'of',
 'the',
 'musical',
 'numbers',
 'are',
 'sped',
 'up',
 'and',
 'rushed',
 'the',
 'show',
 's',
 'hit',
 'song',
 'gets',
 'the',
 'entire',
 'meaning',
 'shattered',
 'when',
 'it',
 'is',
 'given',
 'to',
 'cassie',
 's',
 'character',
 'the',
 'overall',
 'staging',
 'is',
 'very',
 'self',
 'conscious',
 'the',
 'only',
 'reason',
 'i',
 'give',
 'it',
 'a',
 '2',
 'is',
 'because',
 'a',
 'few',
 'of',
 'the',
 'great',
 'numbers',
 'are',
 'still',
 'able',
 'to',
 'be',
 'enjoyed',
 'despite',
 'the',
 '

In [88]:
tokenizer_porter(df.loc[4,'review'])

['i',
 'recent',
 'bought',
 'the',
 'dvd',
 'forget',
 'just',
 'how',
 'much',
 'i',
 'hate',
 'the',
 'movi',
 'version',
 'of',
 'a',
 'choru',
 'line',
 'everi',
 'chang',
 'the',
 'director',
 'attenborough',
 'made',
 'to',
 'the',
 'stori',
 'fail',
 'by',
 'make',
 'the',
 'director',
 'cassi',
 'relationship',
 'so',
 'promin',
 'the',
 'entir',
 'ensembl',
 'premis',
 'of',
 'the',
 'music',
 'sail',
 'out',
 'the',
 'window',
 'some',
 'of',
 'the',
 'music',
 'number',
 'are',
 'sped',
 'up',
 'and',
 'rush',
 'the',
 'show',
 's',
 'hit',
 'song',
 'get',
 'the',
 'entir',
 'mean',
 'shatter',
 'when',
 'it',
 'is',
 'given',
 'to',
 'cassi',
 's',
 'charact',
 'the',
 'overal',
 'stage',
 'is',
 'veri',
 'self',
 'consciou',
 'the',
 'onli',
 'reason',
 'i',
 'give',
 'it',
 'a',
 '2',
 'is',
 'becaus',
 'a',
 'few',
 'of',
 'the',
 'great',
 'number',
 'are',
 'still',
 'abl',
 'to',
 'be',
 'enjoy',
 'despit',
 'the',
 'film',
 's',
 'attempt',
 'to',
 'squeez',
 'ever

In [91]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\himanshu
[nltk_data]     pandey\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [92]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter(df.loc[4,'review']) if w not in stop]

['recent',
 'bought',
 'dvd',
 'forget',
 'much',
 'hate',
 'movi',
 'version',
 'choru',
 'line',
 'everi',
 'chang',
 'director',
 'attenborough',
 'made',
 'stori',
 'fail',
 'make',
 'director',
 'cassi',
 'relationship',
 'promin',
 'entir',
 'ensembl',
 'premis',
 'music',
 'sail',
 'window',
 'music',
 'number',
 'sped',
 'rush',
 'show',
 'hit',
 'song',
 'get',
 'entir',
 'mean',
 'shatter',
 'given',
 'cassi',
 'charact',
 'overal',
 'stage',
 'veri',
 'self',
 'consciou',
 'onli',
 'reason',
 'give',
 '2',
 'becaus',
 'great',
 'number',
 'still',
 'abl',
 'enjoy',
 'despit',
 'film',
 'attempt',
 'squeez',
 'everi',
 'bit',
 'joy',
 'spontan']

##### Transform data to TF-IDF vectors

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer (strip_accents = None,
                        lowercase = False,
                        preprocessor = None,
                        tokenizer = tokenizer_porter,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True)
y=df.sentiment.values
x=tfidf.fit_transform(df.review)

In [100]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.5,random_state=42)

#### Hyperparameter tuning

In [110]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

classifier=LogisticRegressionCV(cv=5,
                       scoring='accuracy',
                       n_jobs=-1,
                       verbose=3,
                       max_iter=300).fit(X_train,Y_train)
saved_model = open('saved_model.sav','wb')
pickle.dump(classifier,saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.3min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.4min finished


### MODEL EVALUATION

In [111]:
filename='saved_model.sav'
saved_classifier=pickle.load(open(filename,'rb'))

In [112]:
saved_classifier.score(X_test,Y_test)

0.89696

###### Accuracy of model comes out to be 89.6 percent